In [6]:
import numpy as np
import rioxarray
from rasterio.enums import Resampling
from osgeo import gdal

In [7]:
mds_file = 'data/raster/MDS_sampa-ZSTD.tif'
mds_file_20 = 'data/raster/MDS_sampa-ZSTD-20m.tif'

In [8]:
ds = gdal.Translate(mds_file_20, mds_file, xRes=20, yRes=20, resampleAlg="average", format='GTiff',  multithread=True)

In [5]:
ds.

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f4f25f87b70> >

In [6]:
mds = rioxarray.open_rasterio('data/raster/MDS_sampa-ZSTD.tif', chunks=1000, lock=False)

In [7]:
mds

,Array,Chunk
Bytes,52.54 GiB,3.81 MiB
Shape,"(1, 146905, 96008)","(1, 1000, 1000)"
Count,14260 Tasks,14259 Chunks
Type,float32,numpy.ndarray


In [41]:
mds_std = 60 # 3 X o STD parece um bom chute para SP

In [42]:
max_horizontal_distance = 10000
max_vertical_distance = 3 * mds_std
thetas = 40
phis = 8
delta_theta = 2*np.pi/thetas
resolution = 0.5
phis_angles = np.arcsin(np.arange(1/phis/2 + 1/phis, 1, 1/phis))
tangents = 1/np.tan(phis_angles) #CoTangente
# indices = np.indices(xds.rio.shape)

In [26]:
pad_max_meters = np.ceil((tangents[::-1] * max_vertical_distance))
pad_max_meters

array([ 67., 130., 191., 265., 370., 548., 943.])

In [35]:
downscales = np.floor(tangents[::-1])
downscales = np.where(downscales == 0, 1, downscales)
downscales

array([1., 1., 1., 1., 2., 3., 5.])

In [38]:
pad_max_meters / downscales

array([ 67.        , 130.        , 191.        , 265.        ,
       185.        , 182.66666667, 188.6       ])

In [11]:
np.rad2deg(phis_angles)

array([10.80692287, 18.20995686, 25.94447977, 34.22886633, 43.43253656,
       54.3409123 , 69.63586519])